In [ ]:
import google.generativeai as genai
import PIL.Image # For image handling
import os
import sys
# Add project root to sys.path to find config
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..')) # Adjust based on notebook location relative to root
if project_root not in sys.path:
    sys.path.append(project_root)
import config # Import the config module

# --- Configuration ---
# API Key is now loaded in config.py
if not config.GOOGLE_API_KEY:
    raise ValueError("API key not found. Please check your .env file and config.py.")

genai.configure(api_key=config.GOOGLE_API_KEY)

# --- Function to Send Image and Prompt ---
# Use default model from config, allow override
def ask_gemini_about_image(image_path: str, text_prompt: str, model_name: str = config.DEFAULT_GEMINI_FLASH_MODEL):
    """
    Sends an image and a text prompt to the specified Gemini model.

    Args:
        image_path: Path to the image file.
        text_prompt: The question or instruction related to the image.
        model_name: The Gemini model to use (defaults to config.DEFAULT_GEMINI_FLASH_MODEL).

    Returns:
        The text response from the model, or None if an error occurs.
    """
    print(f"Loading image from: {image_path}")
    print(f"Using model: {model_name}")
    print(f"Sending prompt: {text_prompt}")

    try:
        # Load the image using PIL
        img = PIL.Image.open(image_path)

        # Select the Gemini model
        model = genai.GenerativeModel(model_name)

        # Send the prompt and image to the model
        response = model.generate_content([text_prompt, img])

        # Handle potential safety blocks or empty responses
        if not response.parts:
             print("Warning: Received an empty response. This might be due to safety filters.")
             print(f"Prompt Feedback: {response.prompt_feedback}")
             return None
        if response.candidates[0].finish_reason.name != "STOP":
             print(f"Warning: Generation finished unexpectedly. Reason: {response.candidates[0].finish_reason.name}")

        return response.text

    except FileNotFoundError:
        print(f"Error: Image file not found at '{image_path}'")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# --- Example Usage ---
if __name__ == "__main__":
    # Use image path from config
    image_file = config.DEFAULT_IMAGE_PATH 

    # The question you want to ask about the image
    prompt = "What objects are in this image? Describe the scene."

    # Choose the model using config defaults
    model_choice = config.DEFAULT_GEMINI_PRO_MODEL # Or use Pro from config

    # Check if the image file exists before calling
    if os.path.exists(image_file):
        result = ask_gemini_about_image(image_file, prompt, model_choice)

        if result:
            print("\n--- Gemini Response ---")
            print(result)
        else:
            print("\nFailed to get a response from Gemini.")
    else:
        print(f"Error: The file '{image_file}' does not exist. Please check config.py.")

Loading image from: D:\LinkGenix\src\notebooks\assests\website-sitemap (1).png
Using model: gemini-1.5-pro-latest
Sending prompt: What objects are in this image? Describe the scene.

--- Gemini Response ---
The image is a flowchart or sitemap, likely for an e-commerce website. It visually represents the different pages and user journeys within the site, particularly focusing on the shopping and classifieds posting processes.  It uses rectangular boxes connected by lines to illustrate the flow and hierarchy of information.

The main categories, branching directly from the "Home" page, are:

* **Shop:** This section deals with the standard purchasing process. It leads to "Product List", then "Product Detail Page", "Cart", and finally "Checkout".  At the cart stage, the flow diverges based on whether the user is "Logged In" or "Not Logged In," with the latter path leading to a "Log In / Register" page.

* **Classifieds:** This branch focuses on posting classified advertisements.  It inclu

In [ ]:
import fitz  # PyMuPDF
import os
import tempfile
from PIL import Image
import io
import sys
# Add project root to sys.path if not already done
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
import config # Import the config module

# Use default DPI from config, allow override
def pdf_to_screenshots_for_spaces(uploaded_pdf, output_dir=None, dpi=config.DEFAULT_PDF_DPI, page_range=None):
    """
    Convert PDF pages to PNG images, designed for Hugging Face Spaces.
    
    Args:
        uploaded_pdf: File object from Streamlit/Gradio upload or bytes data
        output_dir: Optional directory to save files (temp dir used if None)
        dpi: Resolution for the images (defaults to config.DEFAULT_PDF_DPI)
        page_range: Tuple of (start_page, end_page) or None for all pages
        
    Returns:
        List of PIL Image objects and their paths if saved
    """
    # Create temp directory if no output specified
    if output_dir is None:
        output_dir = tempfile.mkdtemp()
        print(f"Created temporary directory: {output_dir}")
    elif not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Handle file object or bytes
    temp_pdf_path = os.path.join(output_dir, "temp_pdf.pdf")
    
    # Save uploaded file to temp location if it's not already a path
    if isinstance(uploaded_pdf, (bytes, io.BytesIO)):
        # It's bytes data
        with open(temp_pdf_path, "wb") as f:
            if isinstance(uploaded_pdf, bytes):
                f.write(uploaded_pdf)
            else:
                f.write(uploaded_pdf.read())
        pdf_path = temp_pdf_path
    elif hasattr(uploaded_pdf, 'read'):
        # It's a file-like object
        with open(temp_pdf_path, "wb") as f:
            f.write(uploaded_pdf.read())
        pdf_path = temp_pdf_path
    else:
        # Assuming it's already a file path
        pdf_path = uploaded_pdf
    
    # Open the PDF
    try:
        pdf_document = fitz.open(pdf_path)
    except Exception as e:
        print(f"Error opening PDF: {e}")
        return []
    
    # Determine page range
    if page_range:
        start_page, end_page = max(0, page_range[0]), min(len(pdf_document)-1, page_range[1])
    else:
        start_page, end_page = 0, len(pdf_document)-1
    
    print(f"Converting pages {start_page+1} to {end_page+1} of PDF")
    
    images = []
    image_paths = []
    
    # Process each page
    for page_num in range(start_page, end_page+1):
        page = pdf_document.load_page(page_num)
        
        # Set the resolution using dpi argument
        zoom = dpi / 72
        matrix = fitz.Matrix(zoom, zoom)
        
        # Create a pixmap (image)
        pixmap = page.get_pixmap(matrix=matrix, alpha=False)
        
        # Save the image
        output_path = os.path.join(output_dir, f"page_{page_num + 1}.png")
        pixmap.save(output_path)
        image_paths.append(output_path)
        
        # Also convert to PIL Image
        img_data = pixmap.tobytes("png")
        img = Image.open(io.BytesIO(img_data))
        images.append(img)
        
        print(f"Processed page {page_num + 1}")
    
    pdf_document.close()
    
    # Clean up temp file if created
    if pdf_path == temp_pdf_path and os.path.exists(temp_pdf_path):
        os.remove(temp_pdf_path)
    
    return images, image_paths


In [ ]:
import os
import sys
# Add project root to sys.path if not already done
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
import config # Import the config module

# Use PDF path from config
pdf_file = config.DEFAULT_PDF_PATH

# Check if the file exists
if not os.path.exists(pdf_file):
    print(f"Error: PDF file not found at '{pdf_file}'. Check config.py.")
else:
    # Call the function with the path from config
    try:
        images, paths = pdf_to_screenshots_for_spaces(pdf_file)
        print(f"Generated {len(images)} images.")
    except NameError:
        print("Error: pdf_to_screenshots_for_spaces function not defined in this scope.")
    except Exception as e:
        print(f"An error occurred during PDF conversion: {e}")

Created temporary directory: C:\Users\SARTHA~1\AppData\Local\Temp\tmpdkw2tcw7
Converting pages 1 to 10 of PDF
Processed page 1
Processed page 2
Processed page 3
Processed page 4
Processed page 5
Processed page 6
Processed page 7
Processed page 8
Processed page 9
Processed page 10


([<PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>,
  <PIL.PngImagePlugin.PngImageFile image mode=RGB size=4000x2250>],
 ['C:\\Users\\SARTHA~1\\AppData\\Local\\Temp\\tmpdkw2tcw7\\page_1.png',
  'C:\\Users\\SARTHA~1\\AppData\\Local\\Temp\\tmpdkw2tcw7\\page_2.png',
  'C:\\Users\\SARTHA~1\\AppData\\Local\\Temp\\tmpdkw2tcw7\\page_3.png',
  'C:\\Users\\SARTHA~1\\AppData\\Local\\Temp\\tmpdkw2tcw7\\page_4.png',
  'C:\\Users\\SARTHA~1\\AppData\\Local\\T

In [ ]:
# This cell remains empty or can be used for further testing


In [2]:
import os
import sys
# Ensure project root is in sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

try:
    # Reload the module to get the updated function
    import importlib
    import src.core.generators.text_generator
    importlib.reload(src.core.generators.text_generator)
    from src.core.generators.text_generator import generate_text_response
    import config # Needed for model selection and API key check within the function
except ImportError as e:
    print(f"Error importing text_generator: {e}")
    print("Ensure the file exists at 'src/core/generators/text_generator.py' and sys.path is correct.")
    generate_text_response = None # Prevent NameError later

if generate_text_response:
    # --- Test Data ---
    test_system_prompt = "You are a helpful assistant that summarizes text concisely."
    test_user_prompt = ("Please summarize the following content in exactly one sentence: \n\n" 
                      "Generative AI models, like those developed by Google and OpenAI, "
                      "are capable of understanding and generating human-like text, code, and images. "
                      "They are trained on vast datasets and can be used for various applications, "
                      "including content creation, translation, and analysis.")

    # --- Call the Function ---
    # Use the default model specified in config.py
    print(f"Testing text generation with model: {config.DEFAULT_GEMINI_PRO_MODEL}")
    try:
        # Call with separate system and user prompts
        response = generate_text_response(test_system_prompt, test_user_prompt)

        # --- Print the Result ---
        if response:
            print("\n--- Text Generator Response ---")
            print(response)
        else:
            print("\nFailed to get a text response.")
    except Exception as e:
         print(f"An error occurred during the test: {e}")

C:\Users\SarthakPandey\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Testing text generation with model: gemini-1.5-pro-latest
Using model: gemini-1.5-pro-latest
Sending prompt: Summarize the following text in one sentence:
Processing content (length: 274 chars)

--- Text Generator Response ---
Generative AI models, trained on massive datasets, can understand and create human-like text, code, and images for diverse applications.


--- Text Generator Response ---
Generative AI models, trained on massive datasets, can understand and create human-like text, code, and images for diverse applications.



In [13]:
import google.generativeai as genai
import os
import sys

# This model name is not a standard publicly documented model accessible
# via the google-generativeai library for image generation with a generic key.
# The code demonstrates the structure, but it will likely fail
# because the model doesn't exist or doesn't support this use case via this API.
MODEL_NAME = 'gemini-experimental-image-gen' # Replace with actual experimental model name if known and accessible

try:
    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("GOOGLE_API_KEY environment variable not set.")
    genai.configure(api_key=api_key)
except ValueError as e:
    print(f"Error: {e}")
    print("Please set the GOOGLE_API_KEY environment variable.")
    sys.exit(1)
except ImportError:
    print("Error: Required libraries not found.")
    print("Please run: pip install google-generativeai")
    sys.exit(1)

def attempt_image_generation(prompt: str):
    try:
        model = genai.GenerativeModel(MODEL_NAME)
        print(f"Calling model '{MODEL_NAME}' with prompt: '{prompt}'")

        # This is the standard method. The response structure for image output
        # from a text prompt is undefined for typical Gemini models via this library.
        response = model.generate_content(prompt)

        # Print the full response object to inspect its structure
        print("\n--- Full Response Object ---")
        print(response)
        print("--------------------------")

        # You would need logic here to parse the response if it
        # hypothetically contained image data (e.g., base64 or bytes).
        # This is highly dependent on the *actual* experimental model's
        # output format, which is not standard for gemini-pro etc.

        try:
            print("\n--- Response Text Content (if any) ---")
            print(response.text)
            print("-------------------------------------")
        except Exception:
            print("\nResponse object does not contain easily accessible text (.text property).")

    except Exception as e:
        print(f"\nAn error occurred: {e}")
        print(f"Make sure the model '{MODEL_NAME}' exists and is accessible with your key for this purpose.")

if __name__ == "__main__":
    test_prompt = "Generate an image of a robot reading a book in a library."
    attempt_image_generation(test_prompt)

Calling model 'gemini-experimental-image-gen' with prompt: 'Generate an image of a robot reading a book in a library.'

An error occurred: 404 models/gemini-experimental-image-gen is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Make sure the model 'gemini-experimental-image-gen' exists and is accessible with your key for this purpose.

An error occurred: 404 models/gemini-experimental-image-gen is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.
Make sure the model 'gemini-experimental-image-gen' exists and is accessible with your key for this purpose.


In [5]:
import requests
import os
import base64
from PIL import Image
import io
import sys

# --- Configuration ---
# Friendli API Credentials
FRIENDLI_TOKEN = "flp_TQQqSUhYVNzgCTO9zXyAJpZWciaDoDm57ECzb1EoNgx68" # Replace with your actual token or load from env
FRIENDLI_TEAM_ID = "L0578lMYXTdk" # Replace with your actual team ID or load from env
FRIENDLI_ENDPOINT_ID = "o4bklt7bj3w6" # Your dedicated endpoint ID
FRIENDLI_ENDPOINT_URL = f"https://api.friendli.ai/dedicated/{FRIENDLI_ENDPOINT_ID}/v1/images/generations" # Updated URL

def generate_image_with_friendli(prompt: str):
    """Generates an image using the Friendli AI dedicated endpoint."""

    headers = {
        "Authorization": f"Bearer {FRIENDLI_TOKEN}", # Added Authorization Header
        "X-Friendli-Team": FRIENDLI_TEAM_ID, # Added Team ID Header
        "Content-Type": "application/json",
    }

    # Payload structure based on reference, using endpoint ID as model
    payload = {
        "model": FRIENDLI_ENDPOINT_ID, # Use endpoint ID as model identifier
        "prompt": prompt,
        "num_inference_steps": 10, # Added based on reference
        "n": 1,  # Kept from previous version
        "size": "1024x1024", # Kept from previous version
        "response_format": "b64_json" # Kept from previous version
        # Add other parameters like 'negative_prompt', 'style_raw' etc. if needed
    }

    print(f"Sending request to: {FRIENDLI_ENDPOINT_URL}")
    print(f"Payload: {payload}")
    print(f"Headers: {{'Authorization': 'Bearer ***', 'X-Friendli-Team': '{FRIENDLI_TEAM_ID}', 'Content-Type': 'application/json'}}") # Print headers safely

    try:
        response = requests.post(FRIENDLI_ENDPOINT_URL, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)

        response_data = response.json()

        # Check the structure of the response - this depends on the API
        if 'data' in response_data and len(response_data['data']) > 0 and 'b64_json' in response_data['data'][0]:
            b64_image_data = response_data['data'][0]['b64_json']
            image_bytes = base64.b64decode(b64_image_data)
            img = Image.open(io.BytesIO(image_bytes))
            print("Image generated successfully!")
            return img
        else:
            print("Error: Unexpected response format from Friendli API.")
            print("Response JSON:", response_data)
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error making request to Friendli API: {e}")
        if hasattr(e, 'response') and e.response is not None:
            print(f"Status Code: {e.response.status_code}")
            try:
                print(f"Response Body: {e.response.json()}")
            except requests.exceptions.JSONDecodeError:
                print(f"Response Body: {e.response.text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# --- Example Usage ---
if __name__ == "__main__": 
    test_prompt = "A futuristic cityscape at sunset, digital art"
    generated_image = generate_image_with_friendli(test_prompt)

    if generated_image:
        # In a notebook, displaying the image is often desired
        # generated_image.show() # Uncomment to display directly if environment supports it
        # Or save it
        try:
            save_path = "friendli_generated_image.png"
            generated_image.save(save_path)
            print(f"Image saved to {os.path.abspath(save_path)}")
        except Exception as e:
            print(f"Error saving image: {e}")

Sending request to: https://api.friendli.ai/dedicated/o4bklt7bj3w6/v1/images/generations
Payload: {'model': 'o4bklt7bj3w6', 'prompt': 'A futuristic cityscape at sunset, digital art', 'num_inference_steps': 10, 'n': 1, 'size': '1024x1024', 'response_format': 'b64_json'}
Headers: {'Authorization': 'Bearer ***', 'X-Friendli-Team': 'L0578lMYXTdk', 'Content-Type': 'application/json'}
Error making request to Friendli API: 404 Client Error: Not Found for url: https://api.friendli.ai/dedicated/o4bklt7bj3w6/v1/images/generations
Status Code: 404
Response Body: {'detail': 'Not Found'}
Error making request to Friendli API: 404 Client Error: Not Found for url: https://api.friendli.ai/dedicated/o4bklt7bj3w6/v1/images/generations
Status Code: 404
Response Body: {'detail': 'Not Found'}
